In [34]:
import pandas as pd
import re
from collections import Counter
from crawl_utils.url_extractor import *
from crawl_utils.table_parser import *
from crawl_utils.html_request import *
from crawl_utils.create_table import *

In [52]:
HSPT = pd.read_csv('HSPT_CODE.csv')

In [35]:
HSPT_URL = pd.read_csv('HSPT_URL.csv')

In [36]:
HSPT_URL.shape

(239, 2)

In [53]:
len(set(HSPT.hspt_name)), len(set(HSPT_URL.hspt_name)), len(set(HSPT_URL.url))

(243, 239, 226)

In [48]:
many_url = {key for key, value in Counter(HSPT_URL.url).items() if value>1}
many_url

{'http://seoul.barunsesang.co.kr/',
 'http://www.ansan21.com/',
 'http://www.djtntn.com/',
 'http://www.hihospital.kr/',
 'http://www.himplus.co.kr/',
 'http://www.nanoori.co.kr/',
 'https://anyspine.co.kr/',
 'https://himchanhospital.com/index.php',
 'https://www.himchanhospital.com/index.php',
 'https://www.seoulchuk.com/main.do'}

In [51]:
for _ in HSPT_URL[HSPT_URL.url.apply(lambda e: e in many_url)].groupby('url'):
    print(_[1])

          hspt_name                              url
93         서울바른세상병원  http://seoul.barunsesang.co.kr/
125  서울바른병원(舊.바른병원)  http://seoul.barunsesang.co.kr/
   hspt_name                      url
9   안산21세기병원  http://www.ansan21.com/
48    21세기병원  http://www.ansan21.com/
    hspt_name                     url
65    글로벌튼튼병원  http://www.djtntn.com/
178      튼튼병원  http://www.djtntn.com/
    hspt_name                        url
92       하이병원  http://www.hihospital.kr/
127    일산하이병원  http://www.hihospital.kr/
    hspt_name                        url
159     플러스병원  http://www.himplus.co.kr/
192    힘플러스병원  http://www.himplus.co.kr/
            hspt_name                        url
6    의료법인나누리의료재단나누리병원  http://www.nanoori.co.kr/
33              나누리병원  http://www.nanoori.co.kr/
104        의료법인 나누리병원  http://www.nanoori.co.kr/
                 hspt_name                      url
17     의료법인 토마스의료재단 윌스기념병원  https://anyspine.co.kr/
126  의료법인 토마스의료재단 안양윌스기념병원  https://anyspine.co.kr/
             hs

In [54]:
diff = set(HSPT_URL.hspt_name).difference(set(pd.read_csv('HSPT_CHILDREN_URL_depth_1.csv').hspt_name))

In [55]:
select_sub_page_by_hspt_list(HSPT_URL, diff).to_csv('cannot_get_sub.csv')

In [56]:
non_non = select_sub_page_by_hspt_list(HSPT_URL, diff)

In [57]:
non_non

,hspt_name,url
4,의)상원의료재단부평힘찬병원,https://himchanhospital.com/index.php
6,의료법인나누리의료재단나누리병원,http://www.nanoori.co.kr/
7,제일정형외과병원,http://www.cheilos.com/
8,나누리수원병원,http://sw.nanoori.co.kr/customer
9,안산21세기병원,http://www.ansan21.com/
...,...,...
228,허리사랑병원,http://www.ilovespine.com/
231,첨단우리병원,http://www.woori-hospital.com/
233,의료법인 백산의료재단 K마디병원,https://www.kmadi.com/home
234,기둥병원,http://www.kidunghospital.net/guide/


In [23]:
result

[('의)상원의료재단부평힘찬병원',
  4    https://himchanhospital.com/index.php
  Name: root_url, dtype: object,
  'js'),
 ('의료법인나누리의료재단나누리병원',
  6    http://www.nanoori.co.kr/
  Name: root_url, dtype: object,
  '#'),
 ('제일정형외과병원',
  7    http://www.cheilos.com/
  Name: root_url, dtype: object,
  'frame'),
 ('나누리수원병원',
  8    http://sw.nanoori.co.kr/customer
  Name: root_url, dtype: object,
  '404'),
 ('안산21세기병원',
  9    http://www.ansan21.com/
  Name: root_url, dtype: object,
  'js'),
 ('의료법인 상원의료재단 부산힘찬병원',
  15    https://himchanhospital.com/index.php
  Name: root_url, dtype: object,
  'js'),
 ('대전우리병원',
  19    http://www.woorispine.com/default/
  Name: root_url, dtype: object,
  'image'),
 ('MS재건병원',
  23    http://www.msrhospital.kr/
  Name: root_url, dtype: object,
  'frame'),
 ('메트로병원',
  27    https://www.metro-suseong.co.kr/main/main.php
  Name: root_url, dtype: object,
  'js'),
 ('마디병원',
  28    http://madi.or.kr/main/ko/index.html
  Name: root_url, dtype: object,
  'js'),
 ('서울척병원',
  30 

In [31]:
hand_len = hand_len = len(list(zip(*result))[0])

In [32]:
# result['text'] = ['비급여_수작업기입' for _ in range(hand_len)]
hand = pd.DataFrame()
hand['whynot'] = list(zip(*result))[2]
hand['url'] = [_.values[0] for _ in list(zip(*result))[1]]
hand['hspt_name'] = list(zip(*result))[0]
hand['date'] = ['200317' for _ in range(hand_len)]

In [60]:
hand.to_csv('why_cannot_get_url.csv')

In [62]:
hand

,whynot,url,hspt_name,date
0,js,https://himchanhospital.com/index.php,의)상원의료재단부평힘찬병원,200317
1,#,http://www.nanoori.co.kr/,의료법인나누리의료재단나누리병원,200317
2,frame,http://www.cheilos.com/,제일정형외과병원,200317
3,404,http://sw.nanoori.co.kr/customer,나누리수원병원,200317
4,js,http://www.ansan21.com/,안산21세기병원,200317
...,...,...,...,...
69,image,http://www.urispine.kr/index.asp,서울우리병원,200317
70,frame,http://www.woori-hospital.com/,첨단우리병원,200317
71,js,https://www.kmadi.com/,의료법인 백산의료재단 K마디병원,200317
72,map > area,http://kidunghospital.net/index.asp,기둥병원,200317


In [14]:
HSPT_CHILDREN_URL.to_csv('HSPT_CHILDREN_URL_depth_1.csv', index=False)